In [1]:
import pandas as pd
import random
import json
import base64

df = pd.read_csv('data/loan_data.csv')
df = df.drop(columns=['loan_status'])
df

,person_income,person_home_ownership,loan_amnt,loan_int_rate,loan_percent_income,previous_loan_defaults_on_file
0,71948.0,3,35000.0,16.02,0.49,0
1,12282.0,2,1000.0,11.14,0.08,1
2,12438.0,0,5500.0,12.87,0.44,0
3,79753.0,3,35000.0,15.23,0.44,0
4,66135.0,3,35000.0,14.27,0.53,0
...,...,...,...,...,...,...
44995,47971.0,3,15000.0,15.66,0.31,0
44996,65800.0,3,9000.0,14.07,0.14,0
44997,56942.0,3,2771.0,10.02,0.05,0
44998,33164.0,3,12000.0,13.23,0.36,0


In [2]:
columns = df.columns.values
random = random.randint(0, len(columns))
features = df.values[random]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'person_income': 12282.0,
 'person_home_ownership': 2.0,
 'loan_amnt': 1000.0,
 'loan_int_rate': 11.14,
 'loan_percent_income': 0.08,
 'previous_loan_defaults_on_file': 1.0}

In [3]:
inputs['person_home_ownership'] = int(inputs['person_home_ownership'])
inputs['previous_loan_defaults_on_file'] = int(inputs['previous_loan_defaults_on_file'])

In [4]:
import tensorflow as tf
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        )
    )

def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        )
    )


def prepare_json(inputs: dict):
    feature_spec = dict()

    for key, values in inputs.items():
        if isinstance(values, int):
            feature_spec[key] = int_feature(values)
        elif isinstance(values, float):
            feature_spec[key] = float_feature(values)

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{
        'examples': {
            'b64': base64.b64encode(example).decode()
        }
    }]

    return json.dumps({
        'signature_name': 'serving_default',
        'instances': result,
    })


In [5]:
import requests
url = "https://loanpredictions-production.up.railway.app/v1/models/loan-model:predict"
def predict(input):
    response = requests.post(url, data=prepare_json(input), headers={"Content-Type": "application/json"})

    predict = response.json()['predictions'][0][0]
    return predict

In [6]:
prediction = predict(inputs)

print(prediction)

0.0
